<h1>Imports</h1>

In [1]:

from mido import MidiFile, MidiTrack, Message
from mido.midifiles import MetaMessage
from os import listdir
import os
import numpy as np
import pandas as pd

<h1>CONSTANTS</h1>
<p> Read Config </p>

In [2]:
from configparser import ConfigParser
config = ConfigParser()

config.read('config.ini')

NR_TRAINING         = int(config.get('PreprocessingParameters', 'NR_TRAINING'))
COMPRESSION_FACTOR  = int(config.get('PreprocessingParameters', 'COMPRESSION_FACTOR'))
CH_KEY              = str(config.get('PreprocessingParameters', 'CH_KEY'))
NUMBER_OCTAVES      = int(config.get('PreprocessingParameters', 'NUMBER_OCTAVES'))
INSTRUMENT          = str(config.get('PreprocessingParameters', 'INSTRUMENT'))
INSTRUMENT_HARMONY  = str(config.get('PreprocessingParameters', 'INSTRUMENT_HARMONY'))
NUMERATOR           = int(config.get('PreprocessingParameters', 'NUMERATOR'))
DENOMINATOR         = int(config.get('PreprocessingParameters', 'DENOMINATOR'))

print(type(NR_TRAINING))

<class 'int'>


In [3]:
'''NR_TRAINING = 2
COMPRESSION_FACTOR = 1
CH_KEY = 'C'
NUMBER_OCTAVES = 1
INSTRUMENT = 'Piano right'
INSTRUMENT_HARMONY = "Piano left"
NUMERATOR = 4
DENOMINATOR = 4
'''

midi_file = 'D:\\Programming\\Musicron-LSTM\\Songs'
mel_train_files = midi_file

<h1>Get longest song duration </h1>

In [4]:
def getTicks(files_dir,nr_training, comp_factor, instr):
    print('Getting Ticks')
    ticks = []
    counter = 0
    #for file_dir in files_dir:
    for file_name in listdir(files_dir):
        file_path = os.path.join(files_dir,file_name)
        print(file_path)
        if counter<nr_training:
        
            print(file_name)
            counter = counter+1
            mid = MidiFile(file_path)                  
            for track in mid.tracks:
                if(track.name==instr):
                    num_ticks = 0          
                    for message in track:
                        if not isinstance(message, MetaMessage):
                           if(message.type=='note_on'):
                                num_ticks += int(message.time/comp_factor)
                        ticks.append(num_ticks)
    #print(int(sum(ticks)/len(ticks)))
    #return int(sum(ticks)/len(ticks))
    return max(ticks)
mel_ticks = getTicks(midi_file, 1, COMPRESSION_FACTOR, instr=INSTRUMENT)

Getting Ticks
D:\Programming\Musicron-LSTM\Songs\chpn-p18.mid
chpn-p18.mid


<h1>Create Piano Roll</h1>

<h>Functions to get and change Key<h>

In [5]:
def changeKey(key, ch_key):
    print('changeKey')
    dict = {'C':48, 'C#':49,'Db':49, 'D':50, 'D#':51,'Eb':51, 'E':52, 'F': 53, 'F#':54,'Gb':54, 'G':55, 'G#':56,'Ab':56, 'A':57, 'A#':58,'Bb':58,'B':59}
    for dict_key, value in dict.items():
        dict[dict_key] = value - 24
    if key!= ch_key:
        transposition = abs(dict[key]-dict[ch_key])
        #print(transposition)
    else:
        transposition = 0
    
    return transposition, dict[ch_key]

def getKey(mid):
    for track in mid.tracks:
        for message in track:
            if message.type == 'key_signature':
                return message.key

<h2>Get note, time, and On/Off(1/0)</h2>
<p>message.time -> In MIDI file tracks, it is used as delta time (in ticks), and it must be a non-negative integer.<p>

In [6]:
def getNoteCurrentOnOf(mid, comp_factor, transpose, type):
    print('getNoteCurrentOnOf')
    getNoteCurrentOnOf = []
    velocity =[]
    for track in mid.tracks:
        if(track.name==type):
            current_time = 0
            for message in track:
                    if not isinstance(message, MetaMessage):
                        current_time += int(message.time/comp_factor)
                        bol = False
                        if (message.type == 'note_on'):
                            if(message.velocity!=0):
                                note_onoff = 1
                                bol = True
                            else:
                                note_onoff = 0
                                bol = True
                        if (message.type == 'note_off'):
                            note_onoff = 0
                            bol = True
                        if (message.type == 'velocity'):
                            print("here")
                            
                        if(bol==True):
                            getNoteCurrentOnOf.append([message.note+(12-transpose), current_time, note_onoff])
                            velocity.append(message.velocity)    
    return getNoteCurrentOnOf

In [7]:
def getNoteStartLength(getNoteCurrentOnOf):
    print(getNoteStartLength)
    note_on_length_array = []
    first_time = False
    aux = 0
    b = [x[1] for x in getNoteCurrentOnOf]
    aux = b[0]
    if aux != 0:
        first_time = True

    for i, message in enumerate(getNoteCurrentOnOf):
        if message[2] == 1: 
            start_time = message[1]
            for event in getNoteCurrentOnOf[i:]: 
                if event[0] == message[0] and event[2] == 0:
                    length = event[1] - start_time
                    break
                
            note_on_length_array.append([message[0], start_time, length])

    if first_time == True:
        i = 0
        for note,timer,length in note_on_length_array:
            timer = timer - aux
            note_on_length_array[i]=[note, timer, length]
            i = i+1
    return note_on_length_array

In [8]:
def createPianoRoll(files_dir, ticks, nr_training, comp_factor, ch_key, number_octaves, type):
    print('Creating PianoRoll Matrix')
    number_notes = number_octaves*12
    piano_roll = np.zeros((nr_training,ticks, number_notes))
    
    piano_roll_velocity = np.zeros((nr_training,ticks, number_notes))
    counter = 0
    for i, file in enumerate(listdir(files_dir)):
        if counter<nr_training:
            counter = counter + 1
            file_path = os.path.join(files_dir,file)
            mid = MidiFile(file_path)
            print(file_path)
            key = getKey(mid)
            print(key)
            
            if key != ch_key:
                print('Transposing %s to %s' % (key, ch_key))
                transpose, low_note = changeKey(key, ch_key)
                high_note = low_note + number_octaves*12
            else:
                print('Same key')
                transpose, low_note = changeKey(key, ch_key)
                high_note = low_note + number_octaves*12
        
            note_time_onoff = getNoteCurrentOnOf(mid, comp_factor, transpose, type)
            note_on_length = getNoteStartLength(note_time_onoff)
            for message in note_on_length:
                if(message[0]<low_note):
                    while(message[0]<low_note):
                        message[0] = message[0] + 12
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)), message[0]-low_note] = 1
                elif(message[0]>=high_note):
                    while(message[0]>=high_note):
                        message[0] = message[0] - 12
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)),message[0]-low_note] = 1
                else:
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)), message[0]-low_note] =  1
          
    return piano_roll, low_note, note_on_length
mel_roll, low_note, note_on_length = createPianoRoll(mel_train_files, mel_ticks, NR_TRAINING, COMPRESSION_FACTOR,CH_KEY, NUMBER_OCTAVES, INSTRUMENT)
mel_roll_harmony, low_note_harmony, note_on_length_harmony = createPianoRoll(mel_train_files, mel_ticks, NR_TRAINING, COMPRESSION_FACTOR,CH_KEY, NUMBER_OCTAVES, INSTRUMENT_HARMONY)

Creating PianoRoll Matrix
D:\Programming\Musicron-LSTM\Songs\chpn-p18.mid
Ab
Same key
changeKey
getNoteCurrentOnOf
<function getNoteStartLength at 0x0000017E8DD195E0>
Creating PianoRoll Matrix
D:\Programming\Musicron-LSTM\Songs\chpn-p18.mid
Ab
Same key
changeKey
getNoteCurrentOnOf
<function getNoteStartLength at 0x0000017E8DD195E0>


<h1>Get Sequence Length</h1>

<h3> We take 1 measure </h3>
<p> 1 measure in 4/4 time at 480 ticks per beat(tpb) </p>
<ul>0. measure n = [Start_line,     End_line] </ul>
<ul>1. measure 1 = [0,              1*4*480 - 1]</ul>
<ul>2. measure 2 = [1*4*480,        2*4*480 - 1] </ul>
<ul>3. measure 3 = [2*4*480,        3*4*480-1]</ul>

<b> Return: dictionary that contains a list of the time signatures and sequence length

In [9]:
# May be of used in the future
'''def timeSignature(mel_train_files, numerator, denominator, note_on_length):
    print('timeSignature')
    num_files = len(mel_train_files)
    time_sig = np.zeros(num_files)
    dict = {}
    for i,file_name in enumerate(listdir(mel_train_files)):
        # print(file_dir)
        file_path = os.path.join(mel_train_files,file_name)
        mid = MidiFile(file_path)   
        ts = []
        sq = []               
        for track in mid.tracks:
            for message in track:
                if(message.type == 'time_signature'):
                    #if message.denominator == 4:
                    if file_name == 'chpn-p18.mid':
                        seq_length = mid.ticks_per_beat * message.numerator
                        sq.append(seq_length)
                        aux = str(message.numerator)+ '//' + str(message.denominator)
                        ts.append(aux)
                        dict[file_name] = (ts, sq, mid.ticks_per_beat)
                #if(message.numerator == numerator and message.denominator == denominator):

     
    return time_sig
 
seq_length = timeSignature(mel_train_files,NUMERATOR,DENOMINATOR, note_on_length) 

seq_length = np.array(note_on_length)'''

#durations = seq_length[:,1]

"def timeSignature(mel_train_files, numerator, denominator, note_on_length):\n    print('timeSignature')\n    num_files = len(mel_train_files)\n    time_sig = np.zeros(num_files)\n    dict = {}\n    for i,file_name in enumerate(listdir(mel_train_files)):\n        # print(file_dir)\n        file_path = os.path.join(mel_train_files,file_name)\n        mid = MidiFile(file_path)   \n        ts = []\n        sq = []               \n        for track in mid.tracks:\n            for message in track:\n                if(message.type == 'time_signature'):\n                    #if message.denominator == 4:\n                    if file_name == 'chpn-p18.mid':\n                        seq_length = mid.ticks_per_beat * message.numerator\n                        sq.append(seq_length)\n                        aux = str(message.numerator)+ '//' + str(message.denominator)\n                        ts.append(aux)\n                        dict[file_name] = (ts, sq, mid.ticks_per_beat)\n                #i

In [10]:
class TimeSignature():
    def __init__(self, numerator, denominator, tpb):
        self._numerator = numerator
        self._denominator = denominator      
        self._tpb = tpb
        
        #self.nr_ticks = nr_ticks
        #self.time_signature_duration = ts_duration
        #self.measure_length = 0
        self._time_signature_duration = 0
        self._ts_duration_ticks = 0
        
    def set_measure_length(self):
        self.measure_length = self._numerator*self._tpb - 1
        
    
    def get_number_of_measures(self):
        pass
    
    def get_start_time_ticks(self):
        pass
    
    def get_end_time_ticks(self):
        pass
        
    def set_ts_end_ticks(self, duration_ticks):
        self._ts_duration_ticks = duration_ticks
        
    #def get_measure_length(self):
        #return self.measure_length
    
    def get_nth_measure_length(self, measure_number):
        return (measure_number-1 * self._numerator * self._tpb, measure_number * self._numerator * self._tpb)
    
    def get_measure_length(self):
        return self._numerator * self._tpb
    
    def get_ts_end_ticks(self):
        return self._ts_duration_ticks
    
    def get_numerator(self):
        return self._numerator

    def get_denominator(self):
        return self._denominator
    
    def get_tpb(self):
        return self._tpb

    
    
    
        
        

In [11]:
def seqLength(mel_train_files, numerator, denominator, note_on_length):
    print('timeSignature')
    num_files = len(mel_train_files)
    time_sig = np.zeros(num_files)
    dict = {}
    for i,file_name in enumerate(listdir(mel_train_files)):
        # print(file_dir)
        file_path = os.path.join(mel_train_files,file_name)
        mid = MidiFile(file_path)   
        counter_ts = 0
        aux_time = 0       
        dict[file_name] =[]       
        for track in mid.tracks:
            for message in track:    
                if(message.type == 'time_signature'):
                    aux_time += message.time
                    if counter_ts > 0:
                        ts_object.set_ts_end_ticks(aux_time - 1)
                        
                    ts_object = TimeSignature(message.numerator, message.denominator, mid.ticks_per_beat)
                    ts_object.set_measure_length()
                    
                    dict[file_name].append(ts_object)
                    counter_ts += 1
                    
                elif (message.type == 'set_tempo'):
                    aux_time += message.time
                #if(message.numerator == numerator and message.denominator == denominator):

        ts_object.set_ts_end_ticks(aux_time - 1)
    return dict
 
seq_length_structure = seqLength(mel_train_files,NUMERATOR,DENOMINATOR, note_on_length) 


timeSignature


In [12]:
'''def seqLength(mel_train_files, numerator, denominator, note_on_length):
    print('timeSignature')
    num_files = len(mel_train_files)
    time_sig = np.zeros(num_files)
    dict = {}
    for i,file_name in enumerate(listdir(mel_train_files)):
        # print(file_dir)
        file_path = os.path.join(mel_train_files,file_name)
        mid = MidiFile(file_path)   
        ts = []
        sq = []
        aux_time = 0              
        for track in mid.tracks:
            for message in track:
                if(message.type == 'time_signature'):
                    aux_time += message.time
                    #if message.numerator == 5:
                    #if file_name == 'chpn-p18.mid':
                    seq_length = mid.ticks_per_beat
                    sq.append(seq_length)
                    aux = str(message.numerator)+ '//' + str(message.denominator)
                    ts.append(aux)
                    #dict[file_name] = (ts, sq, mid.ticks_per_beat)
                    dict[file_name] = (ts, sq)
                elif (message.type == 'get_tempo'):
                    aux_time += message.time
                #if(message.numerator == numerator and message.denominator == denominator):

     
    return dict
    
 
seq_length_structure = seqLength(mel_train_files,NUMERATOR,DENOMINATOR, note_on_length) 
'''

"def seqLength(mel_train_files, numerator, denominator, note_on_length):\n    print('timeSignature')\n    num_files = len(mel_train_files)\n    time_sig = np.zeros(num_files)\n    dict = {}\n    for i,file_name in enumerate(listdir(mel_train_files)):\n        # print(file_dir)\n        file_path = os.path.join(mel_train_files,file_name)\n        mid = MidiFile(file_path)   \n        ts = []\n        sq = []\n        aux_time = 0              \n        for track in mid.tracks:\n            for message in track:\n                if(message.type == 'time_signature'):\n                    aux_time += message.time\n                    #if message.numerator == 5:\n                    #if file_name == 'chpn-p18.mid':\n                    seq_length = mid.ticks_per_beat\n                    sq.append(seq_length)\n                    aux = str(message.numerator)+ '//' + str(message.denominator)\n                    ts.append(aux)\n                    #dict[file_name] = (ts, sq, mid.ticks_per_be

In [13]:
def createNeuralInputs(mel_roll, seq_length):
    X_train = []
    #aux2 = np.empty()
    for (song_idx, song),(s) in zip(enumerate(mel_roll),seq_length):
        measure_start = 0
        measure_end = 0
        nr_measures = 0
        for ts_object in seq_length[s]:
            while measure_end < ts_object.get_ts_end_ticks() and measure_end<song.shape[0]:
                measure_end = measure_end + ts_object.get_measure_length()
                X_train += [song[measure_start:measure_end]]
                measure_start = measure_end 
                #measure_end += 1 #Be careful
                aux = X_train[nr_measures]
                nr_measures += 1
                #aux2 += song[measure_start:measure_end]
                #print(aux[0])
                #print(aux[len(aux)-1])
                #print('here')
               #measure_end += ts_object.measure_length
            #print("here")
        return X_train

            



seq_length = []
Y_train = createNeuralInputs(mel_roll,seq_length_structure)
X_train = createNeuralInputs(mel_roll_harmony,seq_length_structure)
print("here")

here


In [14]:
'''def createNeuralInputs(mel_roll, seq_length):
    X_train = []
    for (song_idx, song),(s) in zip(enumerate(mel_roll),seq_length):
        measure_start = 0
        measure_end = 0
        nr_measures = 0
        for ts_object in seq_length[s]:
            while measure_end < ts_object.get_ts_duration_ticks() and measure_end<song.shape[0]:
                measure_end = measure_end + ts_object.measure_length()
                X_train += [song[measure_start:measure_end]]
                measure_start = measure_end 
                measure_end += 1
                aux = X_train[nr_measures]
                nr_measures += 1
                #print(aux[0])
                #print(aux[len(aux)-1])
                #print('here')
               #measure_end += ts_object.measure_length
            #print("here")
        #print("here")
    return X_train

            



seq_length = []
Y_train = createNeuralInputs(mel_roll,seq_length_structure)
X_train = createNeuralInputs(mel_roll_harmony,seq_length_structure)'''

'def createNeuralInputs(mel_roll, seq_length):\n    X_train = []\n    for (song_idx, song),(s) in zip(enumerate(mel_roll),seq_length):\n        measure_start = 0\n        measure_end = 0\n        nr_measures = 0\n        for ts_object in seq_length[s]:\n            while measure_end < ts_object.get_ts_duration_ticks() and measure_end<song.shape[0]:\n                measure_end = measure_end + ts_object.measure_length()\n                X_train += [song[measure_start:measure_end]]\n                measure_start = measure_end \n                measure_end += 1\n                aux = X_train[nr_measures]\n                nr_measures += 1\n                #print(aux[0])\n                #print(aux[len(aux)-1])\n                #print(\'here\')\n               #measure_end += ts_object.measure_length\n            #print("here")\n        #print("here")\n    return X_train\n\n            \n\n\n\nseq_length = []\nY_train = createNeuralInputs(mel_roll,seq_length_structure)\nX_train = createNeur

In [15]:
from functools import reduce

##seq_length = 1440


In [16]:
#dict = {'C':48, 'C#':49,'Db':49, 'D':50, 'D#':51,'Eb':51, 'E':52, 'F': 53, 'F#':54,'Gb':54, 'G':55, 'G#':56,'Ab':56, 'A':57, 'A#':58,'Bb':58,'B':59}
dict = {'B':47,'C':48, 'C#':49,'D':50, 'D#':51, 'E':52, 'F': 53, 'F#':54, 'G':55, 'G#':56,'A':57, 'A#':58}
#columns= sorted(dict, key=dict.get, reverse=True)
columns = list(dict.keys())

columns = columns * 3
len(columns)
tmp = 2
for i in range(len(columns)):
    if i % 17 == 0:
        tmp += 1
    columns[i] = columns[i] + str(tmp)
#dict = {'C':48, 'C#':49,'Db':49, 'D':50, 'D#':51,'Eb':51, 'E':52, 'F': 53, 'F#':54,'Gb':54, 'G':55, 'G#':56,'Ab':56, 'A':57, 'A#':58,'Bb':58,'B':59}
dict = {'B':47,'C':48, 'C#':49,'D':50, 'D#':51, 'E':52, 'F': 53, 'F#':54, 'G':55, 'G#':56,'A':57, 'A#':58}
#columns= sorted(dict, key=dict.get, reverse=True)
columns = list(dict.keys())

columns = columns * 3
len(columns)
tmp = 2
for i in range(len(columns)):
    if i % 17 == 0:
        tmp += 1
    columns[i] = columns[i] + str(tmp)
aux_columns = columns[columns.index('C3'):]

In [17]:
#measure 3 = [2*4*480 - 1,    3*4*480-1]</ul>
'''start_measure7 = 6*4*480
end_measure7 =  7*4*480
#df = pd.DataFrame(mel_roll[0,4320:8640])
df = pd.DataFrame(mel_roll[0,:])
df.columns = aux_columns[:len(df.columns)]
df.to_csv("input.csv" , index=True)
'''

'start_measure7 = 6*4*480\nend_measure7 =  7*4*480\n#df = pd.DataFrame(mel_roll[0,4320:8640])\ndf = pd.DataFrame(mel_roll[0,:])\ndf.columns = aux_columns[:len(df.columns)]\ndf.to_csv("input.csv" , index=True)\n'

In [18]:
#df.head(100)

In [19]:
from utilities import midi_utilities as mu

here


In [20]:
song_name = list(seq_length_structure.keys())[0]
mu.createMidiFromPianoRoll(seq_length_structure[song_name][0].get_tpb(), Y_train,r'D:\Programming\Musicron-LSTM\Tester-Inputs','Y_train', 1)

In [21]:
song_name = list(seq_length_structure.keys())[0]
mu.createMidiFromPianoRoll(seq_length_structure[song_name][0].get_tpb(), X_train,r'D:\Programming\Musicron-LSTM\Tester-Inputs','X_train', 1)